# Example: Configuration of SBT Experiment

((TODO: In this tutorial we show how to configure an experiment in OpenSBT to test an AEB Agent.))

((TODO: We show the configuration of the experiment for the simulation of an agent in the CarlaSimulation as well in a DummySimulation (no GPU required).))




OpenSBT provides three interface\abstract classes for the integration of SBT components:

- Simulator integration:
    1. Implement the [`simulate`]() method of the [`Simulator`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/simulation/simulator.py) class. In this method a list of scenario instances is passed to the simulator to execute the SUT in the scenarios. 
    2. Convert the simulation output returned by the simulator to generic `Simulator` instance.

    As an example, OpenSBT already provides extensions for CARLA and Prescan Simulator.

- Fitness\Criticality function integration:

    Implement  `eval` method of the `Fitness` (`Critical`) class.

    *Consider*: Several fitness ojectives are specified in one fitness instance, i.e. the eval method return a vector valued output if more than objective is optimized.

- Algorithm integration:
    Integration is done by subclassing `Optimizer` class.
    There are three options for the integration:
    
    - If the algorithm exists in `pymoo`, instantiate it in `run` and assign to `algorithm` 
    - If the algorithm does not exist in pymoo, override `run` and implement the algorithm. The return type should be a `SimulationResult`.
    - Implement new algorithm by sublassing `Algorithm` from pymoo and use first approach for integration.

 - Search space and experiment definition:

    The path to the scenario, the input variables and its bounds are passed to `ADASProblem`. The `simulate` function together with the fitness and criticality function is also passed. Consider following exemplary problem definition:

        ```python
        problem = ADASProblem(
                            problem_name="PedestrianCrossingStartWalk",
                            scenario_path=os.getcwd() + "/scenarios/PedestrianCrossing.xosc",
                            xl=[0.5, 1, 0],
                            xu=[3, 80, 60],
                            simulation_variables=[
                                "PedSpeed",
                                "EgoSpeed",
                                "PedDistance"],
                            fitness_function=FitnessMinDistanceVelocity(),  
                            critical_function=CriticalAdasDistanceVelocity(),
                            simulate_function=CarlaSimulator.simulate,
                            )
        ```                
    
 - Experiment execution:


    - To start search directly in code:

        ```python
        optimizer = MyOptimizer(problem=problem,
                            config=DefaultSearchConfiguration())
        res = optimizer.run()
        res.write_results(results_folder=results_folder, params = optimizer.parameters)
        ```

    - To start search via console:
        
        1. Create new `Experiment`

            ```python
            experiment = Experiment(name="MyNewExperiment",
                                    problem=problem,
                                    algorithm=AlgorithmType.NSGAII, # Registered in AlgorithmType, updated in run.py
                                    search_configuration=DefaultSearchConfiguration() # Search configuration
                                    )
            ```                        
        2. Make sure the algorithm specific switch case section has been added in run.py:

            ```python
            if (...)
              [...]
            else (algorithm == AlgorithmType.NSGAII):
                optimizer = NsgaIIOptimizer(
                                    problem=problem,
                                    config=config)

                res = optimizer.run()
                res.write_results(results_folder=results_folder, params = optimizer.parameters)
            ``` 
        3. To run the created Experiment execute:

            ```bash
            python run.py -e "MyNewExperiment"
            ```

### Usage Example
We describe the usage of the framework by testing the BehaviourAgent (AEB) in the CARLA Simulator for collisions and close to collision situations. The example has been implemented on Ubuntu.

As testing scenario we consider a pedestrian that is crossing the lane of the ego vehicle. The scenario is provided as an OpenSCENARIO 1.2 [file](scenarios/PedestrianCrossing.xosc). We vary the speed of ego, the speed of pedestrian, and the distance to the ego vehicle when the pedestrian starts walking.


### 1. Integrating the Simulator/SUT

To integrate a simulator we need to implement the [simulate]() method of the [`Simulator`]() class. In this method a scenario instance is passed to the simulator to execute the SUT in the scenario.

The implementation of *simulate* is simulator specific. For CARLA we have implemented a [module](https://git.fortiss.org/fortissimo/ff1_testing/ff1_carla), that needs to be called from the simulate method. 

### 2. Implementing a fitness function

To implement a new fitness function we need to implement the `Fitness` class (interface). We implement the eval function in the class, which receives as input a [`SimulationOutput`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/simulation/simulator.py#L40-62) and returns a scalar or vector-valued output.
In our example we have a vector valued output, since as the first objective we want to minimize the distance to the pedestrian, and as the second objective we want to maximize the velocity of the ego vehicle. Additionally, we assign inside the class a name to each objective and declare whether the value is maximized or minimized.


```python
class FitnessMinDistanceVelocity(Fitness):
    @property
    def min_or_max(self):
        return "min", "max"

    @property
    def name(self):
        return "Min distance", "Velocity at min distance"

    def eval(self, simout: SimulationOutput) -> Tuple[float]:
        traceEgo = simout.location["ego"]                     # the ego vehicle has the name "ego"   
        tracePed = simout.location["adversary"]               # the vulnerable actor has the name adversary 

        ind_min_dist = np.argmin(geometric.distPair(traceEgo, tracePed))

        # distance between ego and other object
        distance = np.min(geometric.distPair(traceEgo, tracePed))

        # speed of ego at time of the minimal distance
        speed = simout.speed["ego"][ind_min_dist]

        return (distance, speed)

```

Further we implement a [criticality function](evaluation/critical.py) by implementing the interface class `Critical` to indicate when a scenario is considered fault-revealing/critical. The corresponding *eval* function receives as input the fitness value(s) and the simulation output to declare based on this whether a scenario is critical: (here: when 1) a collision ocurred, 2) min distance < 0.5m or 3) ego velocity > 0 (inverted sign)). 


```python
class CriticalAdasFrontCollisions(Critical):
    def eval(self, vector_fitness, simout: SimulationOutput = None):
        if simout is not None:
            isCollision = simout.otherParams['isCollision']
        else:
            isCollision = None

        if (isCollision == True) or (vector_fitness[0] < 0.5) and (vector_fitness[1] < 0):
            return True
        else:
            return False

```
### 3. Integrating the search algorithm

The search technique is represented by the (abstract) `Optimizer` class.
We instantiate in the init function the search algorithm which has to be an instance of [`Algorithm`](https://github.com/anyoptimization/pymoo/blob/main/pymoo/core/algorithm.py) pymoo. We instantiate [`NSGAII`](https://github.com/anyoptimization/pymoo/blob/main/pymoo/algorithms/moo/nsga2.py#L84) from pymoo:

```python
class NsgaIIOptimizer(Optimizer):

    algorithm_name = "NSGA-II"

    def __init__(self,
                problem: Problem,
                config: SearchConfiguration):

        self.config = config
        self.problem = problem
        self.res = None

        if self.config.prob_mutation is None:
            self.config.prob_mutation = 1 / problem.n_var

        self.parameters = {
            "Population size" : str(config.population_size),
            "Number of generations" : str(config.n_generations),
            "Number of offsprings": str(config.num_offsprings),
            "Crossover probability" : str(config.prob_crossover),
            "Crossover eta" : str(config.eta_crossover),
            "Mutation probability" : str(config.prob_mutation),
            "Mutation eta" : str(config.eta_mutation)
        }

        self.algorithm = NSGA2(
            pop_size=config.population_size,
            n_offsprings=config.num_offsprings,
            sampling=FloatRandomSampling(),
            crossover=SBX(prob=config.prob_crossover, eta=config.eta_crossover),
            mutation=PM(prob=config.prob_mutation, eta=config.eta_mutation),
            eliminate_duplicates=True)

        ''' Prioritize max search time over set maximal number of generations'''
        if config.maximal_execution_time is not None:
            self.termination = get_termination("time", config.maximal_execution_time)
        else:
            self.termination = get_termination("n_gen", config.n_generations)

        self.save_history = True
```

### 4. Defining the problem
 
**Consider: Step 2 and 3 is only required when using the console for experiment execution/modification.**

To define an experiment we do the following:

1. We instantiate `ADASProblem` by defining the scenario - in this example represented by an OpenSCENARIO file - and assigning the search space by defining the variables and their upper (xu) and lower bounds(xl) to be varied. In our example the variables have to be defined in the OpenSCENARIO file as parameters in the [`ParameterDeclaration`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/scenarios/PedestrianCrossing.xosc#L5-11) section.
Then we assign the simulator with the SUT and the fitness/criticality function to be used for the optimization. 
Optional parameters are the simulation time, the sampling time and the approximated evaluation time, as well as a flag to indicate whether to display the native scenario visualization provided by the Simulator.

```python
problem = ADASProblem(
                        problem_name="PedestrianCrossingStartWalk",
                        scenario_path=os.getcwd() + "/scenarios/PedestrianCrossing.xosc",
                        xl=[0.5, 1, 0],
                        xu=[3, 80, 60],
                        simulation_variables=[
                            "PedestrianSpeed",
                            "FinalHostSpeed",
                            "PedestrianEgoDistanceStartWalk"],
                        fitness_function=FitnessMinDistanceVelocityFrontOnly(),  
                        critical_function=CriticalAdasFrontCollisions(),
                        simulate_function=CarlaSimulator.simulate,
                        simulation_time=10,
                        sampling_time=100,
                        approx_eval_time=10,
                        do_visualize = False
                        )
                        
```
2. We create an `Experiment` instance, assigning the name, the problem, the algorithm and the search configuration for the algorithm to be used. 

```python
experiment = Experiment(name="1",
                        problem=problem,
                        algorithm=AlgorithmType.NSGAII,
                        search_configuration=DefaultSearchConfiguration())
```

3. We register the experiment to use it via the console.
```python
experiments_store.register(experiment)
```
### 5. Starting search

- To start search without console

```python
optimizer = NsgaIIOptimizer(
                            problem=problem,
                            config=DefaultSearchConfiguration()
                            )
res = optimizer.run()
res.write_results(results_folder=results_folder, params = optimizer.parameters)
```

- To start search via console

To run the experiment with the name "1" we execute:

```bash
python run.py -e 1
```

We can change experiment parameter as e.g., lower and upper bounds of the search parameters and the search time a using flags:


```bash
python run.py -e 1 -min 0 0 -max 10 2 -m  "FinalHostSpeed" "PedestrianSpeed" -t "01:00:00"
```


### Results

When the search has terminated, results are written into the *results* folder in a folder named as the problem name.

OpenSBT creates the following types of plots:

**Design Space Plot**

<img src="example/results/single/PedestrianCrossingStartWalk/NSGA2/11-01-2023_18-37-58/design_space/FinalHostSpeed_PedestrianEgoDistanceStartWalk.png" alt="Design Space Plot" width="600"/>

Critical regions of the search space are highlighted using classification based on the decision tree algorithm. Other classification techniques, e.g., SVM, KNN can be integrated. The interval borders of the regions are written into a `bounds_regions.csv` as in this [example](example/results/single/PedestrianCrossingStartWalk/NSGA2/11-01-2023_18-37-58/classification/bounds_regions.csv). The corresponding decision tree can be investigated in the file named [tree](example/results/single/PedestrianCrossingStartWalk/NSGA2/11-01-2023_18-37-58/classification/tree.pdf).

**Scenario 2D visualization**

<img src="example/results/single/PedestrianCrossingStartWalk/NSGA2/11-01-2023_18-37-58/gif/0_trajectory.gif" alt="Scenario Visualization" width="600"/>

Traces of the ego vehicle (yellow box) and the adversary (blue circle) are visualized in a 2D .gif animation. The [`SimulationOutput`](simulation/simulator)` can be extended by further state parameters, e.g., environmental information to be plotted in the .gif.

**Objective Space Plot**

<img src="example/results/single/PedestrianCrossingStartWalk/NSGA2/26-01-2023_14-15-14/objective_space/Min%20distance_Velocity%20at%20min%20distance.png" alt="Objective Space Plot" width="600"/>

Following csv. files are generated:

- all_testcases: Contains a list of all evaluated testcases.
- calculation_properties: Experiment setup, as algorithm parameters used for search (e.g. population size, number iterations).
- optimal_testcases: List of the "worst/optimal" testcases.
- summary_results: Information on the performance of the algorithm, e.g., number critical test cases found, ratio |critical test cases|/|all test cases|.

```

